In [1]:
import numpy as np
import matplotlib.pyplot as plt
from numpy import sin, cos, exp

from scipy.integrate import solve_ivp

from typing import Callable

import sim_utils
import apollo_utils

In [2]:
class ApolloGuidanceBankController:   
    def __init__(self, ref_data: apollo_utils.ApolloReferenceData, K: float = 1):
        """K: Over-control gain"""
        self.ref_data = ref_data
        self.K = K
    
    @staticmethod
    def from_file(filename: str):
        """Loads reference data from a file and initializes a new guidance controller"""
        ref_data = apollo_utils.ApolloReferenceData.load(filename)
        return ApolloGuidanceBankController(ref_data)

    def reference_bank_angle(self, t, state, params):
        """Bank angle function for open loop guidance that simply returns
           the reference bank angle"""
        v = state[2]
        if v >= 3500:
            return np.deg2rad(75);
        elif v <= 1500:
            return np.deg2rad(50);
        else:
            return np.deg2rad(50 + (75-50)*(v-1500)/(3500-1500))
    
    def closed_loop_guidance(self, t, state, params):
        h, s, v, gam = state
        
        rho0 = params['rho0']
        beta = params['beta']
        H = params['H']
        
        ref_data_row = self.ref_data.get_row_by_velocity(v)
        
        h_ref, s_ref, v_ref, gam_ref = ref_data_row[1:5]
        F1, F2, F3, D_m_ref = ref_data_row[5:9]
        
        rho = rho0 * exp(-h/H) 
        D_m = rho * v * v / (2 * beta)  # Drag Acceleration (D/m)
        
        # Compute reference bankangle
        phi = self.reference_bank_angle(t, state, params)

        # Add correction based on Apollo guidance algorithm
        phi = phi + self.K * ( -(s-s_ref) - F2*(h-h_ref) + F1*(D_m - D_m_ref ))/F3
        phi = abs(phi)
        phi = max(min(phi, np.pi), 0)
        return phi

# Monte Carlo Simulation

In [6]:
import tqdm.contrib

def monte_carlo_simulation(bank_controller, num_iter=1000):
    """Performs monte carlo simulations with pre-defined initial conditions
       and variances for given bank controller"""
    
    # Initial conditions
    h0 = 120e3; # Entry altitude
    V0 = 5500;  # Entry velocity
    gamma0_deg = -14.5; # Entry flight path angle
    s0 = 0

    # Model parameters
    params = {'H': 11.1e3,
                  'rho0': 0.020, # kg/m^3
                  'beta': 120,
                  'LD': 0.24,
                  'R_m': 3380e3,
                  'g': 3.73}

    # Terminal altitude
    h_f = 20e3

    gamma0 = np.deg2rad(gamma0_deg)

    t0 = 0
    tf = 500.
    tspan = np.linspace(t0, tf, 1001)
    
    # 3-sigma variations to be simulated
    beta_3sigma = 0.05;
    LD_3sigma = 0.08;
    gamma_3sigma= 0.6 * np.pi/180;
    
    # Generate random spreads for parameters and initial conditions
    beta_vec = np.random.normal(params['beta'], beta_3sigma/3, num_iter)
    LD_vec = np.random.normal(params['beta'], beta_3sigma/3, num_iter)
    gamma0_vec = np.random.normal(params['beta'], beta_3sigma/3, num_iter)

    results = []
    for i in tqdm.trange(num_iter):
        params_i = params.copy()
        params_i['beta'] = beta_vec[i]
        params_i['LD'] = LD_vec[i]
        X0 = np.array([h0, s0, V0, gamma0_vec[i]])
        
        traj_i = sim_utils.simulate_entry_trajectory(
            apollo_utils.traj_eom,
            t0, 
            tf,
            X0,
            h_f,
            params,
            bank_controller,
            t_eval=tspan)
        results.append(traj_i)
    
    return results


apollo = ApolloGuidanceBankController.from_file('apollo_data.npz')
# open_loop_results = monte_carlo_simulation(apollo.reference_bank_angle)
closed_loop_results = monte_carlo_simulation(apollo.closed_loop_guidance)

100%|██████████| 1000/1000 [00:35<00:00, 28.27it/s]


In [4]:
# ipywidgets tqdm matplotlib jupyterlab numpy scipy sympy

In [5]:
len(closed_loop_results)

1000